In [ ]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving df_preprocessed.csv to df_preprocessed (1).csv


In [ ]:
df = pd.read_csv('df_preprocessed.csv')

In [ ]:
df.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


Classify data into Moderatry absent  and Excessively absent. if absent for more than 3 hrs is Excessively absent


In [ ]:
# create target value

df['Absenteeism Time in Hours'].median()

3.0

In [ ]:
targets = np.where(df['Absenteeism Time in Hours'] > df['Absenteeism Time in Hours'].median()
, 1, 0 )

In [ ]:
df['Excessively Absent'] = targets

In [ ]:
df.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessively Absent
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [ ]:
# checking number of 1's in df
targets.sum() / targets.shape[0]

#about 46% are 1's and 54 are 0's

0.45571428571428574

In [ ]:
df.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Day of the Week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours',
       'Excessively Absent'], dtype=object)

In [ ]:
#columns_to_scale = ['Month Value',
#       'Day of the Week', 'Transportation Expense', 'Distance to Work',
#       'Age', 'Daily Work Load Average', 'Body Mass Index',
#       'Children', 'Pets']



In [ ]:
# drop Absenteeism Time in Hours and Check point

df_targets = df.drop(['Absenteeism Time in Hours','Month Value','Education','Distance to Work', 'Daily Work Load Average'], axis=1)



In [ ]:
df_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Day of the Week,Transportation Expense,Age,Body Mass Index,Children,Pets,Excessively Absent
0,0,0,0,1,1,289,33,30,2,1,1
1,0,0,0,0,1,118,50,31,1,0,0
2,0,0,0,1,2,179,38,31,0,0,0
3,1,0,0,0,3,279,39,24,2,0,1
4,0,0,0,1,3,289,33,30,2,1,0


In [ ]:
# Inputs for machine learning model 

df_inputs = df_targets.iloc[:,:-1]
df_output = df_targets.iloc[:,-1:]

In [ ]:
df_inputs.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Day of the Week,Transportation Expense,Age,Body Mass Index,Children,Pets
0,0,0,0,1,1,289,33,30,2,1
1,0,0,0,0,1,118,50,31,1,0
2,0,0,0,1,2,179,38,31,0,0
3,1,0,0,0,3,279,39,24,2,0
4,0,0,0,1,3,289,33,30,2,1


In [ ]:
df_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Day of the Week',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Children',
       'Pets'], dtype=object)

In [ ]:
absent_scaler = StandardScaler()

In [ ]:

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
   
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [ ]:
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4']

columns_to_scale = [x for x in df_inputs.columns.values if x not in columns_to_omit]

In [ ]:
columns_to_scale

['Day of the Week',
 'Transportation Expense',
 'Age',
 'Body Mass Index',
 'Children',
 'Pets']

In [ ]:
absent_scaler = CustomScaler(columns_to_scale)

In [ ]:
scaled_inputs = absent_scaler.fit_transform(df_inputs)

In [ ]:
scaled_inputs.shape


(700, 10)

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(scaled_inputs,targets,test_size=0.2, random_state=52)


In [ ]:
X_test.shape , y_test.shape

((140, 10), (140,))

In [ ]:
X_train.shape , y_train.shape

((560, 10), (560,))

In [ ]:
#train model

reg = LogisticRegression()

In [ ]:
reg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# model accuracy 
reg.score(X_train,y_train)

0.7785714285714286

In [ ]:
model_oututs = reg.predict(X_train)
model_oututs

array([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,

In [ ]:
model_oututs == y_train

array([ True, False,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False,  True, False,  True,
        True,  True,  True,  True, False, False,  True,  True,  True,
        True,  True,  True, False,  True,  True, False,  True,  True,
       False, False,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True, False,
        True,  True,

In [ ]:
reg.intercept_, reg.coef_

(array([-1.7493374]),
 array([[ 2.76646024,  0.83685263,  3.22921221,  0.91498816, -0.22054055,
          0.69240603, -0.17566007,  0.28926046,  0.48929789, -0.28604054]]))

In [ ]:
feature_name = df_inputs.columns.values

In [ ]:
summary_table = pd.DataFrame(columns=['Feature name'], data = feature_name)

summary_table['Coefficient'] = np.transpose(reg.coef_)

summary_table

,Feature name,Coefficient
0,Reason_1,2.766460
1,Reason_2,0.836853
2,Reason_3,3.229212
3,Reason_4,0.914988
4,Day of the Week,-0.220541
5,Transportation Expense,0.692406
6,Age,-0.175660
7,Body Mass Index,0.289260
8,Children,0.489298
9,Pets,-0.286041


In [ ]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficient
0,Intercept,-1.749337
1,Reason_1,2.766460
2,Reason_2,0.836853
3,Reason_3,3.229212
4,Reason_4,0.914988
5,Day of the Week,-0.220541
6,Transportation Expense,0.692406
7,Age,-0.175660
8,Body Mass Index,0.289260
9,Children,0.489298


In [ ]:
#exponetials on coefficients 
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)

In [ ]:
summary_table

,Feature name,Coefficient,Odds_ratio
0,Intercept,-1.749337,0.173889
1,Reason_1,2.766460,15.902244
2,Reason_2,0.836853,2.309088
3,Reason_3,3.229212,25.259750
4,Reason_4,0.914988,2.496746
5,Day of the Week,-0.220541,0.802085
6,Transportation Expense,0.692406,1.998518
7,Age,-0.175660,0.838903
8,Body Mass Index,0.289260,1.335440
9,Children,0.489298,1.631171


In [ ]:
summary_table.sort_values('Odds_ratio', ascending=False)

,Feature name,Coefficient,Odds_ratio
3,Reason_3,3.229212,25.259750
1,Reason_1,2.766460,15.902244
4,Reason_4,0.914988,2.496746
2,Reason_2,0.836853,2.309088
6,Transportation Expense,0.692406,1.998518
9,Children,0.489298,1.631171
8,Body Mass Index,0.289260,1.335440
7,Age,-0.175660,0.838903
5,Day of the Week,-0.220541,0.802085
10,Pets,-0.286041,0.751232


Backward elimination - Drop Month Value, Education,Daily Work Load Average as their weights (coefficient) are almost zero

In [ ]:
#Test model 

reg.score(X_test,y_test)

0.7214285714285714

In [ ]:
#prediction probabilities 

pred_prob = reg.predict_proba(X_test)

pred_prob

array([[0.22735272, 0.77264728],
       [0.44860362, 0.55139638],
       [0.32640584, 0.67359416],
       [0.81698869, 0.18301131],
       [0.88048996, 0.11951004],
       [0.71946784, 0.28053216],
       [0.48908446, 0.51091554],
       [0.42402652, 0.57597348],
       [0.73536527, 0.26463473],
       [0.78921132, 0.21078868],
       [0.1318563 , 0.8681437 ],
       [0.4154518 , 0.5845482 ],
       [0.35301273, 0.64698727],
       [0.82489011, 0.17510989],
       [0.24174677, 0.75825323],
       [0.73536527, 0.26463473],
       [0.55323503, 0.44676497],
       [0.30376027, 0.69623973],
       [0.17924352, 0.82075648],
       [0.46157479, 0.53842521],
       [0.77556134, 0.22443866],
       [0.63819179, 0.36180821],
       [0.73536527, 0.26463473],
       [0.60881117, 0.39118883],
       [0.46157479, 0.53842521],
       [0.43191251, 0.56808749],
       [0.2384512 , 0.7615488 ],
       [0.53003451, 0.46996549],
       [0.81294466, 0.18705534],
       [0.91399215, 0.08600785],
       [0.

In [ ]:
pred_prob.shape
# The probabilities referring to q'1
pred_prob[:,1]

array([0.77264728, 0.55139638, 0.67359416, 0.18301131, 0.11951004,
       0.28053216, 0.51091554, 0.57597348, 0.26463473, 0.21078868,
       0.8681437 , 0.5845482 , 0.64698727, 0.17510989, 0.75825323,
       0.26463473, 0.44676497, 0.69623973, 0.82075648, 0.53842521,
       0.22443866, 0.36180821, 0.26463473, 0.39118883, 0.53842521,
       0.56808749, 0.7615488 , 0.46996549, 0.18705534, 0.08600785,
       0.12078353, 0.0807446 , 0.87209702, 0.12078353, 0.62242271,
       0.80737269, 0.91571068, 0.25144808, 0.85452582, 0.64666843,
       0.40588212, 0.79776773, 0.21078868, 0.12849203, 0.68147705,
       0.09252456, 0.43663019, 0.26463473, 0.29770378, 0.16543405,
       0.62242271, 0.89961415, 0.25012144, 0.42721153, 0.59440687,
       0.29266834, 0.15460676, 0.56887056, 0.74244761, 0.12078353,
       0.25144808, 0.10469072, 0.28053216, 0.89335938, 0.61734009,
       0.25012144, 0.88536245, 0.62242271, 0.64666843, 0.75825323,
       0.17577969, 0.21078868, 0.22443866, 0.50086862, 0.94343

In [ ]:
#Save model
import pickle


In [ ]:
with open('model', 'wb') as file:
  pickle.dump(reg, file)

In [ ]:
with open('scaler', 'wb') as file:
  pickle.dump(absent_scaler, file)